In [885]:
#Built using Google Colab
#Data from https://www.basketball-reference.com/leagues/NBA_2020_per_game.html 
#Data from https://www.basketball-reference.com/leagues/NBA_2019_per_game.html 


#Load libraries

%tensorflow_version 2.x 
!pip install -q sklearn

from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

pd.options.display.max_columns = None
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats


get_ipython().run_line_magic('matplotlib', 'inline')
sns.set()
sns.set(rc={'figure.figsize':(12, 8)})

import statsmodels.api as sm

import io
from google.colab import files

In [886]:
#Upload Historical Data
uploaded = files.upload()
train = pd.read_csv(io.BytesIO(uploaded['NBAPrevData.csv']))

Saving NBAPrevData.csv to NBAPrevData (17).csv


In [887]:
#Upload 2020 Data 
uploaded = files.upload()
test =  pd.read_csv(io.BytesIO(uploaded['NBA2020Data.csv']))


Saving NBA2020Data.csv to NBA2020Data (53).csv


In [888]:
train = train[train['G'] > 20] 
print(train.shape)
test= test[test['G'] > 20]
print(test.shape)

(4744, 30)
(450, 30)


In [889]:
print(train.head())
print(test.head)

   Rk                       Player Pos  Age   Tm   G  GS      MP     FG  \
0   1       Álex Abrines\abrinal01  SG   25  OKC  31   2   588.0   56.0   
2   3       Jaylen Adams\adamsja01  PG   22  ATL  34   1   428.0   38.0   
3   4       Steven Adams\adamsst01   C   25  OKC  80  80  2669.0  481.0   
4   5        Bam Adebayo\adebaba01   C   21  MIA  82  28  1913.0  280.0   
7   8  LaMarcus Aldridge\aldrila01   C   33  SAS  81  81  2687.0  684.0   

      FGA    FG%    3P    3PA    3P%     2P     2PA    2P%   eFG%     FT  \
0   157.0  0.357  41.0  127.0  0.323   15.0    30.0  0.500  0.487   12.0   
2   110.0  0.345  25.0   74.0  0.338   13.0    36.0  0.361  0.459    7.0   
3   809.0  0.595   0.0    2.0  0.000  481.0   807.0  0.596  0.595  146.0   
4   486.0  0.576   3.0   15.0  0.200  277.0   471.0  0.588  0.579  166.0   
7  1319.0  0.519  10.0   42.0  0.238  674.0  1277.0  0.528  0.522  349.0   

     FTA    FT%    ORB    DRB    TRB    AST    STL    BLK    TOV     PF  \
0   13.0  0.923  

In [890]:
# train = train.rename(columns={ '3P%': '3PP','2P%':'2PP'})

# train['BLK1'] = (train['BLK']-train['BLK'].min())/(train['BLK'].max()-train['BLK'].min())/train['G']
# train['TRB1'] = (train['TRB']-train['TRB'].min())/(train['TRB'].max()-train['TRB'].min())/train['G']
# train['AST1'] = (train['AST']-train['AST'].min())/(train['AST'].max()-train['AST'].min())/train['G']
# train['DRB1'] = (train['DRB']-train['DRB'].min())/(train['DRB'].max()-train['DRB'].min())/train['G'] 
# train['ORB1'] = (train['ORB']-train['ORB'].min())/(train['ORB'].max()-train['ORB'].min())/train['G']
# train['STL1'] = (train['STL']-train['STL'].min())/(train['STL'].max()-train['STL'].min())/train['G']
# train['STL1'] = (train['STL']-train['STL'].min())/(train['STL'].max()-train['STL'].min())/train['G']
# train['FT1'] = (train['FT%']-train['FT%'].min())/(train['FT%'].max()-train['FT%'].min())/train['G']

train = train.rename(columns={ '3P%': '3PP','2P%':'2PP','FT%':'FTP'})

train['BLK1'] = train['BLK']/train['G']
train['TRB1'] = train['TRB']/train['G']
train['AST1'] = train['AST']/train['G']
train['DRB1'] = train['DRB']/train['G'] 
train['ORB1'] = train['ORB']/train['G']
train['STL1'] = train['STL']/train['G']
train['STL1'] = train['STL']/train['G']


train = train.drop(['Rk','2P','3P','eFG%','FT','FG%','Player','Tm','Age','G','GS','FGA','3PA','2PA','FTA','ORB','DRB','TOV','PF','MP','FG','PTS','BLK','TRB','STL','AST'], axis = 1)

train = train.dropna()

train.head()

,Pos,3PP,2PP,FTP,BLK1,TRB1,AST1,DRB1,ORB1,STL1
0,SG,0.323,0.500,0.923,0.193548,1.548387,0.645161,1.387097,0.161290,0.548387
2,PG,0.338,0.361,0.778,0.147059,1.764706,1.911765,1.441176,0.323529,0.411765
3,C,0.000,0.596,0.500,0.950000,9.500000,1.550000,4.612500,4.887500,1.462500
4,C,0.200,0.588,0.735,0.792683,7.280488,2.243902,5.268293,2.012195,0.865854
7,C,0.238,0.528,0.847,1.320988,9.185185,2.395062,6.086420,3.098765,0.530864


In [892]:
# test = test.rename(columns={'3P%': '3PP','2P%':'2PP'})

# test['BLK1'] = (test['BLK']-test['BLK'].min())/(test['BLK'].max() - test['BLK'].min())/test['G']
# test['TRB1'] = (test['TRB']-test['TRB'].min())/(test['TRB'].max() - test['TRB'].min())/test['G']
# test['AST1'] = (test['AST']-test['AST'].min())/(test['AST'].max() - test['AST'].min())/test['G']
# test['DRB1'] = (test['DRB']-test['DRB'].min())/(test['DRB'].max() - test['DRB'].min())/test['G']
# test['ORB1'] = (test['ORB']-test['ORB'].min())/(test['ORB'].max() - test['ORB'].min())/test['G']
# test['STL1'] = (test['STL']-test['STL'].min())/(test['STL'].max() - test['STL'].min())/test['G']
# test['FT1'] = (test['FT%']-test['FT%'].min())/(test['FT%'].max() - test['FT%'].min())/test['G']

test = test.rename(columns={'3P%': '3PP','2P%':'2PP','FT%':'FTP'})

test['BLK1'] = test['BLK']/test['G']
test['TRB1'] = test['TRB']/test['G']
test['AST1'] = test['AST']/test['G']
test['DRB1'] = test['DRB']/test['G']
test['ORB1'] = test['ORB']/test['G']
test['STL1'] = test['STL']/test['G']


test = test.drop(['Rk','2P','3P','eFG%','FT','FG%','Player','Tm','Age','G','GS','FGA','3PA','2PA','FTA','ORB','DRB','TOV','PF','MP','FG','PTS','BLK','TRB','STL','AST'], axis = 1)

test = test.dropna()

test.head()

,Pos,3PP,2PP,FTP,BLK1,TRB1,AST1,DRB1,ORB1,STL1
0,C,0.333,0.594,0.582,1.063492,9.253968,2.317460,5.968254,3.285714,0.809524
1,PF,0.143,0.564,0.691,1.291667,10.208333,5.111111,7.763889,2.444444,1.138889
2,C,0.389,0.519,0.827,1.641509,7.396226,2.433962,5.452830,1.943396,0.679245
4,SG,0.346,0.391,0.676,0.170213,1.787234,1.893617,1.595745,0.191489,0.361702
5,SG,0.404,0.545,0.867,0.052632,2.236842,1.368421,2.026316,0.210526,0.263158


In [893]:
print(test['Pos'].unique())

['C' 'PF' 'SG' 'SF' 'PG' 'PF-SF' 'SF-SG' 'SF-C' 'SF-PF' 'PF-C' 'SG-SF'
 'SG-PG' 'PG-SG']


In [894]:
#Change names of the weekdays from numbers to actual names
conditions1 = [
    (train['Pos'] == 'SG'),#0
    (train['Pos'] == 'PF') ,#1
    (train['Pos']== 'PG') ,#2
    (train['Pos'] == 'C') ,#3
    (train['Pos'] == 'SF'),#4
    (train['Pos'] == 'PF-SF'),#1
    (train['Pos'] == 'SF-SG'),#4
    (train['Pos'] == 'SG-PF'),#0
    (train['Pos'] == 'C-PF'),#3
    (train['Pos'] == 'SG-SF'),#0
    (train['Pos'] == 'PF-C')#1
    ]

## create a list of the values we want to assign for each condition
#values1 = [1,3,0,4,2,5,6,7,8,6,8]
values1 = [0,1,2,3,4,1,4,0,3,0,1]

## create a new column and use np.select to assign values to it using our lists as arguments
train['Pos1'] = np.select(conditions1, values1)

#Change names of the weekdays from numbers to actual names
conditions2 = [
    (test['Pos'] == 'SG'),#0
    (test['Pos'] == 'PF') ,#1
    (test['Pos']== 'PG') ,#2
    (test['Pos'] == 'C') ,#3
    (test['Pos'] == 'SF'),#4
    (test['Pos'] == 'PF-SF'),#1
    (test['Pos'] == 'SF-SG'),#4
    (test['Pos'] == 'SG-PF'),#0
    (test['Pos'] == 'C-PF'),#3
    (test['Pos'] == 'SG-SF'),#0
    (test['Pos'] == 'PF-C'),#1
    (test['Pos'] == 'SF-C'),#4
    (test['Pos'] == 'SF-PF'),#4
    (test['Pos'] == 'PG-SG'),#2
    (test['Pos'] == 'SG-PG')#0

    ]

## create a list of the values we want to assign for each condition
#values2 = [1,3,0,4,2,5,6,7,8,6,8,8,7,6,6]
values2 = [0,1,2,3,4,1,4,0,3,0,1,4,4,2,0]

## create a new column and use np.select to assign values to it using our lists as arguments
test['Pos1'] = np.select(conditions2, values2)







In [895]:
train = train.drop(['Pos'], axis = 1)
train.head()

,3PP,2PP,FTP,BLK1,TRB1,AST1,DRB1,ORB1,STL1,Pos1
0,0.323,0.500,0.923,0.193548,1.548387,0.645161,1.387097,0.161290,0.548387,0
2,0.338,0.361,0.778,0.147059,1.764706,1.911765,1.441176,0.323529,0.411765,2
3,0.000,0.596,0.500,0.950000,9.500000,1.550000,4.612500,4.887500,1.462500,3
4,0.200,0.588,0.735,0.792683,7.280488,2.243902,5.268293,2.012195,0.865854,3
7,0.238,0.528,0.847,1.320988,9.185185,2.395062,6.086420,3.098765,0.530864,3


In [896]:
test = test.drop(['Pos'], axis = 1)
test.head()


,3PP,2PP,FTP,BLK1,TRB1,AST1,DRB1,ORB1,STL1,Pos1
0,0.333,0.594,0.582,1.063492,9.253968,2.317460,5.968254,3.285714,0.809524,3
1,0.143,0.564,0.691,1.291667,10.208333,5.111111,7.763889,2.444444,1.138889,1
2,0.389,0.519,0.827,1.641509,7.396226,2.433962,5.452830,1.943396,0.679245,3
4,0.346,0.391,0.676,0.170213,1.787234,1.893617,1.595745,0.191489,0.361702,0
5,0.404,0.545,0.867,0.052632,2.236842,1.368421,2.026316,0.210526,0.263158,0


In [897]:
train_y = train.pop('Pos1')
test_y = test.pop('Pos1')
train.head()

,3PP,2PP,FTP,BLK1,TRB1,AST1,DRB1,ORB1,STL1
0,0.323,0.500,0.923,0.193548,1.548387,0.645161,1.387097,0.161290,0.548387
2,0.338,0.361,0.778,0.147059,1.764706,1.911765,1.441176,0.323529,0.411765
3,0.000,0.596,0.500,0.950000,9.500000,1.550000,4.612500,4.887500,1.462500
4,0.200,0.588,0.735,0.792683,7.280488,2.243902,5.268293,2.012195,0.865854
7,0.238,0.528,0.847,1.320988,9.185185,2.395062,6.086420,3.098765,0.530864


In [898]:
print(train_y.unique())
print(test_y.unique())
print('train',train.isnull().sum())
print(test.isnull().sum())

[0 2 3 1 4]
[3 1 0 4 2]
train 3PP     0
2PP     0
FTP     0
BLK1    0
TRB1    0
AST1    0
DRB1    0
ORB1    0
STL1    0
dtype: int64
3PP     0
2PP     0
FTP     0
BLK1    0
TRB1    0
AST1    0
DRB1    0
ORB1    0
STL1    0
dtype: int64


In [899]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)
 

In [900]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))  

In [901]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 9 classes.
    n_classes=9)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmprlvyu1xf', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [902]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmprlvyu1xf/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 2.4486256, step = 0
INFO:tensorflow:global_step/sec: 224.51
INFO:tensorflow:loss = 2.191092, step = 100 (0.451 sec)
INFO:tensorflow:global_step/sec: 298.697
INFO:tensorflow:loss = 2.1781127, step = 200 (0.331 sec)
INFO:tensorflow:global_step/sec: 278.144
INFO:tensorflow:loss = 2.1769726, step = 300 (0.359 sec)
INFO:tensorflow:global_step/sec: 299.023
INFO:tensorflow:loss = 2.1686215, step = 400 (0.334 sec)
INFO:tensorflow:global_step/sec: 266.131
INFO:tensorflow:loss = 2.1670964, step = 500 (0.377 sec)
INFO:ten

In [903]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-01-19T06:23:23Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmprlvyu1xf/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.29765s
INFO:tensorflow:Finished evaluation at 2021-01-19-06:23:24
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.44622424, average_loss = 1.6277769, global_step = 5000, loss = 1.6208911
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmprlvyu1xf/model.ckpt-5000

Test set accuracy: 0.446

